# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key



### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [66]:
output_data = "../Python_Challenges_WeatherPy/Output/weather_data.csv"
# output_data_file

data = pd.read_csv(output_data)

weather_data = data.drop(columns=["Unnamed: 0"])
reduced_weather_data = weather_data.head(10)

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [6]:
gmaps.configure(api_key=g_key)

In [68]:
locations = reduced_weather_data[["Lat", "Lng"]]
# locations

humidity = reduced_weather_data["Humidity"].astype(float)
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [70]:
reduce_data = pd.DataFrame(reduced_weather_data, columns = ["City", "Max Temp", "Wind Speed", "Cloudiness"])
# A max temperature lower than 80 degrees but higher than 70.
new_max_temp = (reduce_data["Max Temp"] <= 80) & (reduce_data["Max Temp"] > 70)
# Wind speed less than 10 mph.
new_wind_speed = reduce_data["Wind Speed"] < 10
# Zero cloudiness.
new_cloudiness = reduce_data["Cloudiness"] == 0
# Drop any rows that don't contain all three conditions. 
new_data = reduce_data
new_data[new_max_temp & new_wind_speed & new_cloudiness]
new_data

,City,Max Temp,Wind Speed,Cloudiness
0,jamestown,48.99,5.82,90
1,new norfolk,72.59,1.61,65
2,buarcos,60.01,5.82,40
3,port blair,72.84,5.21,24
4,christchurch,64.00,10.29,17
5,cape town,55.40,6.93,6
6,atuona,77.86,15.82,80
7,puerto ayora,64.40,14.99,40
8,hobart,61.00,5.82,40
9,miles city,59.00,4.50,75


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [72]:
hotel_df = pd.DataFrame(reduced_weather_data, columns=["City", "Country", "Lat", "Lng"])

hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
0,jamestown,US,42.10,-79.24,
1,new norfolk,LR,6.65,-10.59,
2,buarcos,PT,40.17,-8.88,
3,port blair,CD,-7.03,29.76,
4,christchurch,NZ,-43.53,172.63,
5,cape town,CL,-36.41,-71.90,
6,atuona,PF,-9.80,-139.03,
7,puerto ayora,ES,36.59,-6.23,
8,hobart,AU,-42.88,147.33,
9,miles city,US,35.19,-120.70,


In [73]:
hotel_name = []
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

# taken from Airpost_Ratings Example done in class
print(f"----------------------\nRetrieving Results\n----------------------")
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    city = row['City']
    lat = row["Lat"]
    lng = row["Lng"]
    # change location each iteration while leaving original params in place
    
    params["location"] = f"{lat},{lng}"

    # make request
    response = requests.get(base_url, params=params).json()
    try:
        print(f"The first hotel in {city} is {response['results'][0]['name']}.")
        hotel_name.append(response['results'][0]['name'])
    except IndexError:
        print(f"----------------------Missing field/result... skipping.----------------------")
        hotel_name.append(np.nan)
print(f"----------------------\nRetrieving Complete\n----------------------")

----------------------
Retrieving Results
----------------------
The first hotel in jamestown is Chautauqua Harbor Hotel.
----------------------Missing field/result... skipping.----------------------
The first hotel in buarcos is Hotel Atlântida Sol.
----------------------Missing field/result... skipping.----------------------
The first hotel in christchurch is The Salisbury Hotel.
The first hotel in cape town is Hostal Molino Viejo.
The first hotel in atuona is Hôtel Hiva Oa Hanakee Lodge.
The first hotel in puerto ayora is Pestana.
The first hotel in hobart is The Henry Jones Art Hotel.
The first hotel in miles city is The Cliffs Hotel and Spa.
----------------------
Retrieving Complete
----------------------


In [74]:
hotel_df["Hotel Name"]=hotel_name
hotel_df

,City,Country,Lat,Lng,Hotel Name
0,jamestown,US,42.10,-79.24,Chautauqua Harbor Hotel
1,new norfolk,LR,6.65,-10.59,NaN
2,buarcos,PT,40.17,-8.88,Hotel Atlântida Sol
3,port blair,CD,-7.03,29.76,NaN
4,christchurch,NZ,-43.53,172.63,The Salisbury Hotel
5,cape town,CL,-36.41,-71.90,Hostal Molino Viejo
6,atuona,PF,-9.80,-139.03,Hôtel Hiva Oa Hanakee Lodge
7,puerto ayora,ES,36.59,-6.23,Pestana
8,hobart,AU,-42.88,147.33,The Henry Jones Art Hotel
9,miles city,US,35.19,-120.70,The Cliffs Hotel and Spa


In [75]:
# NOTE: Do not change any of the code in this cell
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [76]:

# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(markers)
fig

# Display figure


Figure(layout=FigureLayout(height='420px'))